In [27]:
import teradata as teradata
import pandas as pd
import getpass
import sys
from tqdm import tqdm
import time

print('Enter details for Teradata connection')
b = input("User ID:")
c = getpass.getpass()

def Teradata_Dev_Conn() :
 # TeraData Connection... 
    udaExec = teradata.UdaExec()  # keep udaexec.ini configuration file in the same directory of python script    
    session= udaExec.connect(method="odbc", system="TDDEV", username=b, password=c, driver="Teradata",authentication='LDAP');
    return session
 
def Teradata_Prod_Conn() :
 # TeraData Connection... 
    udaExec = teradata.UdaExec()  # keep udaexec.ini configuration file in the same directory of python script    
    session= udaExec.connect(method="odbc", system="TDPROD", username=b, password=c, driver="Teradata",authentication='LDAP');
    return session

def Teradata_Stage_Conn() :
 # TeraData Connection... 
    udaExec = teradata.UdaExec()  # keep udaexec.ini configuration file in the same directory of python script    
    session= udaExec.connect(method="odbc", system="TDTEST", username=b, password=c, driver="Teradata",authentication='LDAP');
    return session
choice = input("Enter to which database you would want to connect.(Choose among 1,2,3)\nMake sure your query has correct database name.\n Prod -->  1\n Stage -->  2\n Dev -->  3\n")

if choice=='1':
    connection=Teradata_Prod_Conn()
elif choice=='2':
    connection=Teradata_Stage_Conn()
elif choice=='3':
    connection=Teradata_Dev_Conn()
else:
    print("Invalid Choice.")
    sys.exit()


import os
def get_query():
    d=os.getcwd()
    for filename in os.listdir(d):
        if filename in ("query.txt"):
            f = open(os.path.join(d, filename), "r")
            searchlines = f.readlines()
            total_list = []
            for line in (searchlines):
                total_list.append(line)
            
    
            def listToString(s):
                str1 = " "
                return (str1.join(s))
            query1 = listToString(total_list)
    f.close()
    return(query1)
#query1 = get_query()

query1="""SELECT a.job_grp_name,a.JOB_NAME,a.SQ_INSTANCE_NAME,a.TARGET_INSTANCE_NAME,ltrim(b.target_db_name,'$$') as target_db,a.TARGET_TABLE_NAME,a.TGT_LOAD_ORDER_NO,a.STMT_ATTR_DESC,a.STMT_LINE_NO,
x'0A'||'/*#\#####'||x'0A'|| a.TECH_REPO_NAME || '::'|| a.JOB_GRP_NAME  ||'::'|| a.JOB_NAME  || '::'|| CAST(a.JOB_LAST_SAVED_DTTM AS VARCHAR(20)) ||'::'||  a.JOB_TASK_MAP_NAME || x'0A'
|| '::'|| a.TARGET_INSTANCE_NAME || '::'||a.TARGET_TABLE_NAME || '::'|| TRIM(a.TGT_LOAD_ORDER_NO) || '::'||a.STMT_ATTR_DESC || '::'||TRIM(a.STMT_LINE_NO) || '::' ||x'0A' || '###### */' ||x'0A'
||  a.STMT_TEXT || x'0A' || '/*^^^^^^^^^^^^^*/'||x'0A' AS SQL_TEXT
from bidsmetricsvwdb.job_task_statements a
left join
(sel c.job_name,c.target_instance_name,c. target_db_name, c.target_table_name from bidsmetricsvwdb.JOB_SRC_TGTS  c
group by c.job_name,c.target_instance_name,c. target_db_name, c.target_table_name) b
on a.job_name=b.job_name
and a.target_table_name = b.target_table_name
and a.target_instance_name = b.target_instance_name
where 1=1
and a.tech_repo_name IN('EDW_Prod_101' )
and (--STMT_TEXT LIKE '%WI_AS_UNBILLED_RSTD_BKGS%' --or 
STMT_TEXT LIKE '%MT_RSTD_BKGS_MEASURE%'


--STMT_TEXT LIKE '%WI_MEDIUM_RPT_RSTD%' or 
--STMT_TEXT LIKE '%WI_SALES_HIER_DR_FINHIST%' or 
--STMT_TEXT LIKE '%MT_RR_MTRCS_DTL_FACT%' or 
--STMT_TEXT LIKE '%MT_RSTD_SLS_STK_RR_MTRCS%' or 
--STMT_TEXT LIKE '%MT_RSTD_SUBSCR_METRIC_AMRTZ%' or 
--STMT_TEXT LIKE '%WI_MT_RSTD_SUBSCR_METRIC_AMRTZ%' or 
--STMT_TEXT LIKE '%W_RR_MTRCS_DTL_FACT%' 
)
and target_db <>'NULL'"""


Exec_start_Time=time.time();
try:
    print("Please wait... Connection is getting Established.")
    df = pd.read_sql(query1,connection)
except:
    print("Error in executing the query. Please verify !!! ");
    time.sleep(5)
    sys.exit()
Exec_End_Time=time.time()
print("Your query is executed.")

Total_Exec_Time=Exec_End_Time-Exec_start_Time
print("\n\nTotal Time Taken execute the query and load into dataframe: %f secs"%Total_Exec_Time);
'''
def insert_func(df):
    df=df.fillna('NULL')
    l=[]
    s=""
    value_list=[]
    count=range(0,df.shape[0])
    for ind in tqdm(count):
            
        for i,j in df.items():
            #print(j[0])
            val=j[ind]
            if val is not None:
                try:
                    val=val.strip()
                except:
                    try:
                        val=val.strftime("%Y-%m-%d %H:%M:%S")
                    except:
                        pass
            l.append(val)
        s=""
        st=""
        for i in l:
            if i is not None:
                if i is 'NULL':
                    s=s+"NULL"+","
                else:
                    i=str(i)
                    s=s+"'"+i+"',"
        st="("+s[:-1]+")"
        
        string_value=""
        for i,j in df.items():
            string_value=string_value+i+','
        value="insert into etlvwdb_ts1.dw_job_streams("+string_value[:-1]+") values"+st+";"+"-"*100
        value_list.append(value)
        l=[]
        s=""
        #print("--"*50)
    return(value_list)

if df.shape[0]!=0:
    print("Insert Statements preparation started.")
    final=insert_func(df)
    print("Insert statements prepared. File being created")
    text_file = open("Insert_statements.txt", "w")
else:
    print("Error!!! Please Check the query and database you selected")
    time.sleep(5)
    sys.exit()
try:
    for i in final:
        text_file.write(str(i))
        text_file.write('\n')
except:
    print("Error in writing a file.")
text_file.close()
'''
df
connection.close()
print("Successfully created the file!!")
time.sleep(10)


Enter details for Teradata connection
User ID:vkadimis
········
Enter to which database you would want to connect.(Choose among 1,2,3)
Make sure your query has correct database name.
 Prod -->  1
 Stage -->  2
 Dev -->  3
1
Please wait... Connection is getting Established.
Your query is executed.


Total Time Taken execute the query and load into dataframe: 96.618675 secs
Successfully created the file!!


In [42]:
text_file = open("bids_check.txt", "w")
try:
    for i in set(table_list):
        text_file.write(str(i))
        text_file.write('\n')
except:
    print("Error in writing a file.")
text_file.close()

In [28]:
df

,JOB_GRP_NAME,JOB_NAME,SQ_INSTANCE_NAME,TARGET_INSTANCE_NAME,target_db,TARGET_TABLE_NAME,TGT_LOAD_ORDER_NO,STMT_ATTR_DESC,STMT_LINE_NO,SQL_TEXT
0,EDWTD_BKG_AGGRS,wf_MT_RSTD_BKGS_MSR_ANNUITY_PRDT_HIST,SQ_2_HST_MT_RSTD_BKGS_MEASURE_PRDT,HST_MT_RSTD_BKGS_MEASURE_PRDT2,MGRSTGDB,HST_MT_RSTD_BKGS_MEASURE_PRDT,2.0,SESS_TGT_POST_SQL,1.0,\n/*#\#####\nEDW_Prod_101::EDWTD_BKG_AGGRS::wf...
1,EDWTD_BKG_AGGRS,wf_MT_RSTD_BKGS_MSR_ANNUITY_HIST_09_10,SQ_3_WI_RSTD_BKGS_MSR_ANNUITY_23,WI_RSTD_BKGS_MSR_ANNUITY_23,STGDB,WI_RSTD_BKGS_MSR_ANNUITY_23,3.0,SESS_TGT_POST_SQL,1.0,\n/*#\#####\nEDW_Prod_101::EDWTD_BKG_AGGRS::wf...
2,EDWTD_SALES_ORDERS,wf_MT_DCV_SECURITY_BKGS_MEASURE_3,SQ_WI_DCV_SB_END_CUST_6,WI_DCV_SB_END_CUST_6,STGDB,WI_DCV_SB_END_CUST_6,6.0,SESS_TGT_POST_SQL,1.0,\n/*#\#####\nEDW_Prod_101::EDWTD_SALES_ORDERS:...
3,EDWTD_CA_SERVICES,wf_MT_SERVICES_BOOKINGS_RSTD,SQ_WI_MT_SERVICES_BOOKINGS_RSTD_11,WI_MT_SERVICES_BOOKINGS_RSTD_11,STGDB,WI_MT_SERVICES_BOOKINGS_RSTD,11.0,SESS_SQ,1.0,\n/*#\#####\nEDW_Prod_101::EDWTD_CA_SERVICES::...
4,EDWTD_BKG_AGGRS,wf_MT_RSTD_BKGS_MSR_ANNUITY_MY_TRP,SQ_3_WI_RSTD_BKGS_MSR_ANTY_MY_TRP,WI_RSTD_BKGS_MSR_ANTY_MY_TRP_1,STGDB,WI_RSTD_BKGS_MSR_ANTY_MY_TRP,3.0,MAP_SQ,1.0,\n/*#\#####\nEDW_Prod_101::EDWTD_BKG_AGGRS::wf...
...,...,...,...,...,...,...,...,...,...,...
755,EDWTD_XAAS_20140909,wf_WI_MRR_SUB_ITEMS_UPB1,SQ_WI_MRR_SUB_ITEMS_UPB3,WI_MRR_SUB_ITEMS_UPB31,STGDB,WI_MRR_SUB_ITEMS_UPB3,3.0,SESS_TGT_POST_SQL,1.0,\n/*#\#####\nEDW_Prod_101::EDWTD_XAAS_20140909...
756,EDWTD_BKG_AGGRS,wf_MT_PRODUCT_HWSW_ANNUITY,SQ_WI_PRODUCT_HWSW_ANNUITY_TMP1,WI_PRODUCT_HWSW_ANNUITY_TMP11,STGDB,WI_PRODUCT_HWSW_ANNUITY_TMP1,1.0,MAP_SQ,2.0,\n/*#\#####\nEDW_Prod_101::EDWTD_BKG_AGGRS::wf...
757,EDWTD_XAAS_20140909,wf_WI_MRR_SUB_ITEMS_UPB1,SQ_WI_MRR_SUB_ITEMS_UPB3,WI_MRR_SUB_ITEMS_UPB31,STGDB,WI_MRR_SUB_ITEMS_UPB3,3.0,MAP_SQ,1.0,\n/*#\#####\nEDW_Prod_101::EDWTD_XAAS_20140909...
758,EDWTD_BKG_AGGRS,wf_MT_PRODUCT_HWSW_ANNUITY,SQ_WI_PRODUCT_HWSW_ANNUITY_TMP1,WI_PRODUCT_HWSW_ANNUITY_TMP11,STGDB,WI_PRODUCT_HWSW_ANNUITY_TMP1,1.0,SESS_SQ,2.0,\n/*#\#####\nEDW_Prod_101::EDWTD_BKG_AGGRS::wf...


In [39]:
table_list=[]
for ind,j in enumerate(df['SQL_TEXT']):
    if "sales_rep" in j.lower() or "sales_terr" in j.lower():
        table_list.append(df['TARGET_TABLE_NAME'][ind])
print(len(set(table_list)))

133
